# Introduction to NLP fundamentals in Tensorflow

NLP has the goal of deriving information out of natural langugae (could be sequences, text or speech).

Another common term for NLP problems is seuence to sequence problems (seq2seq).


## Check for GPU

In [2]:
# !nvidia-smi -L

## Get helper functions

In [3]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-11-21 17:05:52--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-11-21 17:05:53 (46.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataset

The dataset we're going to be using is Kaggle's introduction to NLP dataset (text samples of tweets labelled as disaster or not disaster).

Dataset: https://www.kaggle.com/c/nlp-getting-started/data

In [4]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

# unzip data
unzip_data("nlp_getting_started.zip")

--2021-11-21 17:05:56--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.135.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.005s  

2021-11-21 17:05:56 (108 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualize our text samples, we first have to read them in. One way to do so would be to use python: https://realpython.com/read-write-files-python/

Another way to do this is to use pandas.

In [5]:
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [6]:
train_df["text"][10]

'Three people died from the heat wave so far'

In [7]:
# shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [8]:
train_df_shuffled["text"].iloc[3]

'Aftershock back to school kick off was great. I want to thank everyone for making it possible. What a great night.'

In [9]:
# what does the dataframe look like?
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# how many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
# how many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# Let's visualize some random training examples
import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples
for row in train_df_shuffled[["text", "target"]][random_index:random_index+5].itertuples():
  _, text, target = row
  print(f"Target: {target}", "(real disaster)" if target> 0 else "(not real disaster)")
  print(f"Text:{text}\n")
  print("----\n")

Target: 0 (not real disaster)
Text:RT NotExplained: The only known image of infamous hijacker D.B. Cooper. http://t.co/JlzK2HdeTG

----

Target: 0 (not real disaster)
Text:Read this already in '14 but it was and remains one of my favorite articles ever. ?'LetÛªs Like Demolish Laundry'? http://t.co/6suPThAece

----

Target: 0 (not real disaster)
Text:@chojo everyone hail to the pumpkin king!!!!

----

Target: 0 (not real disaster)
Text:'McFadden Reportedly to Test Hamstring Thursday' via @TeamStream http://t.co/jWq4KvJH2j

----

Target: 1 (real disaster)
Text:w--=-=-=-[ NEMA warns Nigerians to prepare for drought http://t.co/5uoOPhSqU3

----



### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split


In [14]:
# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1,
                                                                            random_state=42)

In [15]:
# check the lengths
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [16]:
# check the first 10 samples
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

Wheb dealing with a text problem, one of the first thing we have to do is convert text to numbers.

There are a few ways to do this, namely:  
* **Tokenization** - direct mapping of a token (a token could be a word or a character) to number (text vectorization).
* **Embedding** - create a matrix of feature vector for each token (the size of the feature vector can be defined and this embedding can be learned).

### Text vectorization (Tokenization)

In [17]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [18]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# # Use the default text vectorization parameters
# text_vectorizer = TextVectorization(max_tokens=None, # how many words in the vocabulary (automatically add <OOV>)
#                                     standardize="lower_and_strip_punctuation",
#                                     split="whitespace",
#                                     ngrams=None, # creates groups of n words
#                                     output_mode="int",
#                                     output_sequence_length=None, # how long do you want your sequences to be?
#                                     pad_to_max_tokens=True)


In [19]:
train_sentences[0]

'@mogacola @zamtriossu i screamed after hitting tweet'

In [20]:
train_sentences[0].split()

['@mogacola', '@zamtriossu', 'i', 'screamed', 'after', 'hitting', 'tweet']

In [21]:
len(train_sentences[0].split())

7

In [22]:
# find the average number of tokens (words) in the training tweets
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

Average number of tokens (words) in a tweet is 15.

In [23]:
# Setup text vectorization variables
max_vocab_length = 10000 # max number of words to have in our vocabulary
max_length = 15  # max length our sequences will be (e.g how many words from a tweet our model will see)

text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)


In [24]:
# fit the text vectorizer to training data
text_vectorizer.adapt(train_sentences)

In [25]:
# create a sample sentence and tokenize it
sample_sentence = "There's a flood in my street!"
text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [26]:
# Choose a random sentence from the training dataset and tokenize it
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\n Vectorized version:")
text_vectorizer([random_sentence])

Original text:
 4 dead dozens injured in Gaza blast near house leveled in summer war - Washington Post  http://t.co/AXXDCaKzTY #World        

 Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 178,  115, 1871,  243,    4, 1849,  557,  217,  318, 2898,    4,
         270,  122,  973,  371]])>

In [27]:
# Get the unique words in the vocabulary
words_in_vocab = text_vectorizer.get_vocabulary()  # get all of the unique words in the our training data
top_5_words = words_in_vocab[:5] # get the most common words 
bottom_5_words = words_in_vocab[-5:] # get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f'5 most common words: {top_5_words}')
print(f"5 least common words: {bottom_5_words}")

Number of words in vocab: 10000
5 most common words: ['', '[UNK]', 'the', 'a', 'in']
5 least common words: ['pages', 'paeds', 'pads', 'padres', 'paddytomlinson1']


[UNK] - unknown token - stands for unknown (word out of the vocabulary)

### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use Tensorflow's embedding layer.

The parameters we care most about for our embedding layer:    
* `input_dim` = the size of our vocabulary
* ` output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vector 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [28]:
from tensorflow.keras import layers

embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128,
                             input_length=max_length)

embedding

In [29]:
# Get a random sentence from the training set
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\
        \n\nEmbedded version:")

# embed the random sentence (turn positive indexes into dense vectors of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 A Santa Ana Police Union Blocks Video of the Raid at Sky High Holistic Pot Dispensary - The Atlantic http://t.co/yvV1RlghfT #SmartNews        

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[-0.00744901, -0.03911621,  0.00680882, ...,  0.04167098,
         -0.01193986, -0.00945824],
        [ 0.02153117, -0.04489986, -0.01851456, ...,  0.01071848,
         -0.0442574 , -0.03481395],
        [-0.03052774,  0.02120392,  0.0228928 , ..., -0.0059789 ,
          0.03624513, -0.02510378],
        ...,
        [-0.01707082, -0.00209092, -0.02353652, ..., -0.00498907,
         -0.03505756,  0.01977389],
        [-0.03052774,  0.02120392,  0.0228928 , ..., -0.0059789 ,
          0.03624513, -0.02510378],
        [-0.02227497, -0.00017006, -0.00238796, ...,  0.00081221,
         -0.01346689,  0.00237132]]], dtype=float32)>

In [30]:
# check out a single token's embedding
sample_embed[0][0], sample_embed[0][0].shape, random_sentence.split()[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([-0.00744901, -0.03911621,  0.00680882,  0.04973067,  0.03977503,
        -0.0233068 , -0.04623299, -0.0070407 , -0.04311899,  0.02795695,
        -0.00571625, -0.0042361 , -0.01051427, -0.0393381 ,  0.0377797 ,
         0.00529381, -0.04002118,  0.02412404, -0.0444991 , -0.00335251,
        -0.00881862,  0.04843423, -0.02874802, -0.00747166,  0.04411553,
        -0.01310252, -0.00934488, -0.04873268, -0.00170075, -0.0124191 ,
        -0.02441304, -0.0090358 , -0.03254768, -0.04467125,  0.03949479,
         0.04028857, -0.02650099, -0.02986341,  0.03252354,  0.0049695 ,
         0.02275233, -0.04541706, -0.03238316,  0.00725911,  0.03069297,
        -0.03190857, -0.02923958,  0.03881924,  0.04264433, -0.00692656,
         0.04062385, -0.02804741,  0.03736431, -0.02400316,  0.02580135,
         0.02662541,  0.01002412, -0.04682172,  0.03062682,  0.03708139,
        -0.01447474, -0.01641464, -0.03463502,  0.02978997,  0.02973903,
  

## Modelling a text dataset

Now we've got a way to turn our tetxt sequences into numbers, it's time to start building a series of modelling experiments.

Let;s start with a baseline and move on from there.

 * Model 0: Naive Bayes (baseline) 
 * Model 1: Feed-forward neural network
 * Model 2: LSTM model (RNN)
 * Model 3: GRU model (RNN)
 * Model 4: Bidirectional-LSTM model (RNN)
 * Model 5: 1D Convolutional Neural network (CNN)
 * Model 6: Tensorflow Hub pretrained feature extractor (using transfer learning for NLP)
 * Model 7: Same as model 6 with 10% of training data

We will use the standard steps in modelling with tensorflow:    
* Create a model
* Build a model
* Fit the model
* Evaluate the model






### Model 0: Getting a Baseline Model

As with all machine elarning modelling experiments, it's important to create a baseline model so you've gto a benchmark for future experiments to build upon.

To create our baseline, we'll sue Sklearn's Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers.


In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# create tokenization and modelling pipeline
model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()), # convert words to numbers using tfidf
                    ("clf", MultinomialNB()) # model the text
])

model_0.fit(train_sentences, train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [32]:
# evaluate our baseline model
baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of :{baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of :79.27%


In [33]:
train_df.target.value_counts() # slightly imbalanced classes

0    4342
1    3271
Name: target, dtype: int64

In [34]:
# make predictions
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:10]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0])

### Creating an evaluation function for our model experiments

Let's create a function to compare our model's predictions with the truth label using the following metrics:   
* Accuracy
* Precision
* Recall
* F1-score

In [35]:
# Function to evaluate: accuracy , precision, recall , f1-score
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  '''
  Calculates model accuracy, precision, recall and f1-score of a binary classification model.
  '''
  # calculat model accuracy
  model_accuracy = accuracy_score(y_true, y_pred)*100
  # calculate model precision, recall and f1-score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average="weighted")
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results 

In [36]:
# get baseline results
baseline_results = calculate_results(y_true = val_labels,
                                     y_pred = baseline_preds)
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [37]:
# Create a tensorboard callback (need to create a new one for each model)
from helper_functions import create_tensorboard_callback

# Create a directory to save tensorboard logs
SAVE_DIR = "model_logs"

# Build the model with functional API
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string) # inputs are 1-dimensional strings
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x)  # create an embedding of numberized inputs
x = layers.GlobalAveragePooling1D()(x)  # condense the feature vector for each token to one vector
outputs = layers.Dense(1, activation="sigmoid")(x)  # create the output layer, want binary outputs so use sigmoid activation function
model_1 = tf.keras.Model(inputs, outputs, name="model_1_dense")

In [38]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [39]:
# Compile the model
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

# fit the model
model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences, val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20211121-170558
Epoch 1/5
215/215 [==============================] - 5s 17ms/step - loss: 0.6093 - accuracy: 0.6936 - val_loss: 0.5350 - val_accuracy: 0.7598
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.4413 - accuracy: 0.8159 - val_loss: 0.4689 - val_accuracy: 0.7848
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.3476 - accuracy: 0.8612 - val_loss: 0.4604 - val_accuracy: 0.7900
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.2853 - accuracy: 0.8910 - val_loss: 0.4658 - val_accuracy: 0.7861
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.2383 - accuracy: 0.9094 - val_loss: 0.4779 - val_accuracy: 0.7822


In [40]:
# check the results
model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 2ms/step - loss: 0.4779 - accuracy: 0.7822


[0.4779238700866699, 0.7821522355079651]

In [41]:
#make some predictions and evaluate those
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [42]:
# look at a single prediction
model_1_pred_probs[0]

array([0.40781426], dtype=float32)

In [43]:
# Look  at the first 10 predictions
model_1_pred_probs[:10]

array([[0.40781426],
       [0.74169064],
       [0.9978651 ],
       [0.13736346],
       [0.12324706],
       [0.9407549 ],
       [0.91622734],
       [0.9923202 ],
       [0.9638504 ],
       [0.2918291 ]], dtype=float32)

In [44]:
# convert model prediction probabilities to label format
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))
model_1_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 0.], dtype=float32)>

In [45]:
# calculate model_1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

{'accuracy': 78.21522309711287,
 'f1': 0.779380517102918,
 'precision': 0.7860403042839984,
 'recall': 0.7821522309711286}

In [46]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [47]:
import numpy as np
np.array(list(model_1_results.values())) >  np.array(list(baseline_results.values()))

array([False, False, False, False])

Comparing results shows that baseline model outperforms our first deep leanring based dense model.


## Visualizing learned embeddings

In [48]:
# Get the vocabulary from the text vectorization layer
words_in_vocab = text_vectorizer.get_vocabulary()

In [49]:
len(words_in_vocab)

10000

In [50]:
# first 10 most common words
words_in_vocab[:10]

['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is']

In [51]:
max_vocab_length

10000

In [52]:
# Model 1 summary
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, 15)               0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [53]:
# get the weight matrix of embedding layer
# (these are the numerical representations of each token in our training data which have been learned for 5 epochs)
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights

array([[-0.0631176 , -0.01131928,  0.03073541, ...,  0.04005573,
        -0.0146791 , -0.0098086 ],
       [-0.04242303,  0.01067005,  0.01291454, ...,  0.00507351,
         0.04119867, -0.03608679],
       [ 0.01476165, -0.00292486,  0.00118733, ...,  0.0303147 ,
         0.0449791 ,  0.01551725],
       ...,
       [-0.00779443, -0.0221359 ,  0.03439063, ...,  0.02775896,
        -0.04755497, -0.00384204],
       [-0.01197284, -0.04332977, -0.04595361, ..., -0.00999974,
         0.0348996 , -0.02159029],
       [-0.02141334, -0.05398636, -0.10672767, ...,  0.03988824,
         0.04226118, -0.03685109]], dtype=float32)

In [54]:
print(embed_weights.shape) # same size as vocab size and embedding_dim (output_dim of embedding layer)

(10000, 128)


Now we've got the embedding matrix our model has learned to represent our tokens, let's see how we can visualize it:    

To do so, Tensorflow has a handy tool called **Projector**: http://projector.tensorflow.org/

And TensorFlow also has an incredible guide on **word embeddings** themselves: https://www.tensorflow.org/text/guide/word_embeddings

In [55]:
# create embedding files (we've got this from Tensorflow's word embeddings documentation)
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [56]:
# # Download files from colab to upload to projector
# try:
#   from google.colab import files
#   files.download('vectors.tsv')
#   files.download('metadata.tsv')
# except Exception:
#   pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNNs are useful for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the representation of a later input.

- MIT deep learning: http://introtodeeplearning.com/
- Andrej Karpathy's blog post: http://karpathy.github.io/2015/05/21/rnn-effectiveness/

### Model 2 - LSTM

LSTM stands for **Long short term memory** (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:    
```
Input (text) -> Tokenize -> Embedding -> Layers (RNN/Dense) -> Output (label probability)
``` 

In [58]:
# Create an LSTM model

from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
x = layers.LSTM(64, return_sequences=True)(x)
print(x.shape)
x = layers.LSTM(64)(x)
print(x.shape)
x = layers.Dense(64, activation="relu")(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs, outputs, name="model_2_LSTM") 

(None, 15, 128)
(None, 15, 64)
(None, 64)
